In [ ]:
#imported libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os
import rasterio
import numpy as np
from rasterio.transform import from_origin
from rasterio import features
import shapely
from rasterio.features import rasterize
from rasterio.enums import MergeAlg
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap

In [ ]:
# create the shape file for the wards
shapefile_dir = 'Data\LB_shp'
gdfs = []
for file in os.listdir(shapefile_dir):
    if file.endswith('.shp'):
        filepath = os.path.join(shapefile_dir, file)
        gdf = gpd.read_file(filepath)
        gdfs.append(gdf)

merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))


In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('Data/2022-01/2022-01-metropolitan-street.csv')

df = df[['Month','Longitude','Latitude','Crime type']]

#filter all burgleies
burglaries = df[df['Crime type']=="Burglary"]
b_gdf = gpd.GeoDataFrame(
    burglaries, 
    geometry=gpd.points_from_xy(burglaries.Longitude, burglaries.Latitude),
)
#correct css
b_gdf.set_crs(epsg=4326, inplace=True) 
b_gdf = b_gdf.to_crs(epsg=27700)



In [ ]:
#method to get all points in a certain ward and set the value equal to a column value of that ward
def neighborhood_amount(points,wards,col_name):
    points_with_polygons = gpd.sjoin(points, wards, how="inner")
    point_counts = points_with_polygons.groupby("index_right").size()
    wards[col_name] = wards.index.map(point_counts).fillna(0)
    return(wards)

In [ ]:
#loop to do this method for each dataset

## example here with only burglaries
wards= (neighborhood_amount(b_gdf,merged_gdf,"burglaries"))
wards.explore(column="burglaries", cmap="YlGnBu", legend=True)

## GWR Implementation

In [ ]:
#assign predicting variable
y = wards['burglaries'].values.reshape((-1,1))
#assign factors
X = wards[["streetlights","policestations","pricepersize"]].values

#definition of coordiantes
u = gdf['X']
v = gdf['Y']
coords = list(zip(u,v))

In [ ]:
%%time
gwr_selector = Sel_BW(coords, y, X)
gwr_bw = gwr_selector.search()
#determine bandwith with bi-square kernel and AIC kriteria
print('GWR bandwidth =', gwr_bw)

In [ ]:
#fit the model
gwr_results = GWR(coords, y, X, gwr_bw).fit()
gwr_results.summary()

https://deepnote.com/app/carlos-mendez/PYTHON-GWR-and-MGWR-71dd8ba9-a3ea-4d28-9b20-41cc8a282b7a

link handles GWR, if it compiles until here, rest can be taken from here as well

# Old Code

In [ ]:
#get he dataset from the crime dataset
def merge_datasets(ranges_year):
    dataset_path_data =""
    timestep = 0
    df =pd.DataFrame()
    for i in range(ranges_year[0],ranges_year[1]+1):
        for j in range(1,13):
            timestep+=1
            if len(str(j))<2:
                dataset_path_data = str(i)+'-0'+str(j)
            else:
                dataset_path_data = str(i)+'-'+str(j)
            temp_df = pd.read_csv('Data/'+dataset_path_data+'/'+dataset_path_data+'-metropolitan-street.csv')
            temp_df = temp_df[['Month','Longitude','Latitude','Crime type']]
            temp_df['timestep']= timestep
            df = pd.concat([df,temp_df],ignore_index=True)
            
    temp_gdf = gpd.GeoDataFrame(
        df, 
        geometry=gpd.points_from_xy(df.Longitude, df.Latitude),
    )
    temp_gdf.set_crs(epsg=4326, inplace=True)  # First assign WGS84 CRS
    temp_gdf = temp_gdf.to_crs(epsg=27700)
    return(temp_gdf)
            

In [ ]:
merged_gdf_1 = merged_gdf.unary_union
merged_gdf_2= merged_gdf_1.exterior
outer_boundary_gdf = gpd.GeoDataFrame(geometry=[merged_gdf_2], crs=merged_gdf.crs)
outer_boundary_gdf.explore()

In [ ]:

# Read the CSV file into a DataFrame
df = pd.read_csv('Data/2022-01/2022-01-metropolitan-street.csv')

df = df[['Month','Longitude','Latitude','Crime type']]
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.Longitude, df.Latitude),
)
gdf.set_crs(epsg=4326, inplace=True)  # First assign WGS84 CRS
gdf = gdf.to_crs(epsg=27700)

# Clip the gdf to the boundary
clipped_gdf = gpd.clip(gdf, merged_gdf)

In [ ]:
clipped_gdf = gpd.clip(merge_datasets([2022,2024]), merged_gdf)

In [ ]:
def rasters_overtime(cell_size,crime_types):

    filtered_to_type_df = clipped_gdf[clipped_gdf['Crime type'].isin(crime_types)]


    # Get the total bounds
    bounds = clipped_gdf.total_bounds
    width = int((bounds[2] - bounds[0]) / cell_size)
    height = int((bounds[3] - bounds[1]) / cell_size)

    rasters = []
        
    tf = from_origin(bounds[0], bounds[3], cell_size, cell_size)
    for i in filtered_to_type_df['timestep'].unique():
        timestep_gdf = filtered_to_type_df[filtered_to_type_df['timestep']==i]
        points = []
        for geom in timestep_gdf.geometry:
            if isinstance(geom, shapely.geometry.point.Point):
                points.append((geom,1))
            else:
                print(f"Skipping non-Point geometry: {geom}")


        # Rasterize: each point will contribute to its corresponding pixel
        t_raster = features.rasterize(
            shapes=points,
            out_shape=(height, width),
            fill= 0,
            transform=tf,
            merge_alg= MergeAlg.add, 
            dtype="int32",
        )
        rasters.append(t_raster)
        
    return(rasters)

In [ ]:
full_entries = []
rasters_to_animate = rasters_overtime(150,['Robbery'])
for i in rasters_to_animate:
    full_entries.extend(i.tolist())
full_entries= pd.Series(full_entries)
full_entries = full_entries[full_entries!=0]

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.matshow(rasters_to_animate[0], cmap="viridis")  # Initial raster plot
fig.colorbar(cax)
outer_boundary_gdf.boundary.plot(ax=ax, color='yellow', linewidth=2)  # Adjust color and linewidth as needed

# Update function for the animation
def update(frame):
    cax.set_array(rasters_to_animate[frame])
    ax.set_title(f"Time Step: {frame + 1}")  # Update the title with the frame

# Create the animation
ani = FuncAnimation(fig, update, frames=len(rasters_to_animate), interval=500)

# To display the animation
plt.show()

# To save the animation as a GIF or MP4
ani.save("raster_animation.gif", writer="imagemagick")  # Requires ImageMagick installed
# ani.save("raster_animation.mp4", writer="ffmpeg")  # Requires ffmpeg installed
